# 시계열 데이터 이상탐지란

이상치란 **'다른 방법에 의해 생성되었다는 의심이 될 정도로 관찰에서 너무 많이 벗어난 관찰이다.'** 따라서 이상값이란 예상되는 동작에 따르지 않는 관측치로 볼 수 있다.

![Outlier](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Time-series-outliers.png?ssl=1)

시계열에서 이상값은 두 가지 다른 의미를 갖는다. 원치않은 데이터일 경우와 특정 사건에 의한 데이터이다.

</br></br>

시계열 데이터에서 이러한 이상값 탐지에 대표적인 예시는 *사기 탐지*다. 주요 목표가 이상치 자체를 탐지하고 분석하는 것이다. 시계열에서 이상감지는 'Point Outlier', 'Subsequence Outlier' 두 유형으로 분류 할 수 있다.

![Outlie2](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Time-series-outliers-types.png?resize=587%2C377&ssl=1)

</br>

# Point Outlier
Point Outlier는 시계열의 다른 값 또는 인접한 값들과 비교할 때 특정 시간에서 비정상적인 데이터를 말한다.

![Point Outlier](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Point-outliers-in-time-series.png?resize=750%2C301&ssl=1)

</br>

# Subsequence Outlier
Subsequence Outlier는 같은 움직임이 비정상적인 시간의 연속적인 현상을 의미한다. 전역 또는 로컬일 수 있으며 하나 또는 그 이상 시간 종속 변수에 영향을 줄 수도 있다.

![Ss Outlier](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/%E2%80%8ASubsequence-outliers-in-time-series.png?ssl=1)

</br></br></br>

## 시계열 데이터의 이상 감지 기술

### STL 분해
STL(Seasonal and Trend decomposition using Loess) Loess는 비선형 관계를 추정하기 위한 기법이다. 

![STL](https://otexts.com/fppkr/fpp_files/figure-html/elecequip-stl2-1.png)


STL에서는 t.window(Trend), s.window(Seasonal)의 매개변수에 따라 추세-주기와 계절성을 조정할 수 있다. 

</br></br>

### 분류 및 회귀 트리(CART)

의사결정 나무의 훌륭한 분류 기능을 통해 시계열 데이터에서의 이상값을 식별할 수 있다. 그 방법은

- 의사결정 나무 모델에 이상과 비이상을 분류할 수 있는 포인트를 학습 시킬 수 있다. 이를 위해서 이상치 데이터에 대한 레이블링이 필요하고 이 데이터는 토이 데이터셋 외부에서는 쉽게 발견할 수 없어야 한다.

- 학습시키지 않은 데이터도 필요하다. Isolation Forest 알고리즘을 사용하여 레이블이 지정된 데이터 집합의 도움 없이 특정 포인트가 이상치인지 아닌지 예측할 수 있다.

핵심은 Isolation Forest는 다른 이상값 검색 방법과 달리 일반 데이터를 프로파일링 하는 대신에 명시적으로 이상치를 식별한다. Isolation Forest는 다른 앙상블 기법과 마찬가지로 의사결정 트리를 기반으로 한다. 즉, Isolation Forest는 약간의 다른 데이터 포인트라는 사실을 기반으로 이상치를 감지한다. 또한, 거리 또는 밀도 측정을 사용하지 않고 동작한다.

- Isolation Forest 모델을 만들때 (contamination = outliers_fraction)옵션을 세팅하는데 이는 현재 데이터에 이상값의 비율을 모델에 알려주는 것입니다. 이는 trial/error 메트릭스이다.
- 데이터에 대한 학습과 예측을 진행하면 정상에 대해서는 1을 이상치에 대해서는 -1을 반환한다.
- 마지막으로 이상치를 시계열로 시각화한다.

~~~
plt.rc('figure',figsize=(12,6))
plt.rc('font',size=15)
catfish_sales.plot()
~~~


![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Time-series-anomalies.png?ssl=1)


~~~
#전처리
outliers_fraction = float(.01)
scaler = StandardScaler()
np_scaled = scaler.fit_transform(catfish_sales.values.reshape(-1, 1))
data = pd.DataFrame(np_scaled)

# 모델 학습
model =  IsolationForest(contamination=outliers_fraction)
model.fit(data)


catfish_sales['anomaly'] = model.predict(data)

# 시각화
fig, ax = plt.subplots(figsize=(10,6))
a = catfish_sales.loc[catfish_sales['anomaly'] == -1, ['Total']] #이상치
ax.plot(catfish_sales.index, catfish_sales['Total'], color='black', label = 'Normal')
ax.scatter(a.index,a['Total'], color='red', label = 'Anomaly')
plt.legend()
plt.show();
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-Detection-Isolation-Forest.png?ssl=1)

그림을 보면 굉장히 아상치에 탐지를 잘 했음을 알 수 있다. 하지만 시자 지점에서의 이상치 탐지가 좋지 못한 결과같아 보인다. 이에 대한 이유는 두 가지가 있다.
</br>
- 처음에는 알고리즘이 이상치로 판단할 수 있는 데이터가 많지 않기 떄문인데 이는 많은 데이터를 얻을수록 더 많은 분산을 볼 수 있게 되고 스스로 그 범위를 조정하기 때문
- 이런 값들이 많이 보인다면 모델의 contamination 파라미터를 너무 높게 설정했기 때문이다.
</br>

</br>

**장점**</br>
이 기술의 가장 큰 이점은 더욱 정교한 모델을 만들기 위해서 원하는 만큼 많은 변수나 피처를 넣을 수 있다는 것입니다.


**단점**</br>
단점은 피처가 증가함에 따라서 컴퓨터 퍼포먼스에 영향을 준다. 이럴 경우 피처 선택을 신중하게 해야합니다.

</br></br></br>

### 예측을 사용한 이상치 탐지
미래 예측을 통한 이상치 감지는 과거의 데이터에 white noise를 추가하여 미래에 대한 예측을 생성하는 접근 방식이다. 미래에 대한 예측에 대한 선은 부드러운 형태일 것입니다.</br>
이 방법을 사용할때 어려운 점은 차이의 수, 자기회귀 계수, 예측 오류 계수를 선택해야한다는 것입니다.
</br>
이는 새로운 신호의 작업을 할때마다 새로운 예측 모델을 만들어야합니다.
</br>
또 다른 장애요소는 차이를 확인한 후 신호가 고정되어 있어야 한다는 것입니다. 간단히 말해, 신호가 시간에 의존해서는 안됩니다. 이는 중요한 제약 조건입니다.
</br>
이동평균(Moving Average), 자기회귀(Autoregress) 접근 방식 및 ARIMA와 같은 방법으로 활용할 수 있습니다. ARIMA를 통한 이상 탐지 방법은 아래와 같다.
- 과거 데이터를 통해 예측과 훈련 데이터와의 크기 차이가 있다.
- 임계값(theshold)를 설정하고 임계값과 다름을 기준으로 이상치를 식별한다.
</br>

이를 위해 Prophet 모듈을 사용합니다.

~~~
def fit_predict_model(dataframe, interval_width = 0.99, changepoint_range = 0.8):
   m = Prophet(daily_seasonality = False, yearly_seasonality = False, weekly_seasonality = False,
               seasonality_mode = 'additive',
               interval_width = interval_width,
               changepoint_range = changepoint_range)
   m = m.fit(dataframe)
   forecast = m.predict(dataframe)
   forecast['fact'] = dataframe['y'].reset_index(drop = True)
   return forecast

pred = fit_predict_model(t)
~~~

아래 코드는 Prophet에서 제공하는 interval_width를 가져오는 과정입니다.

~~~
def detect_anomalies(forecast):
   forecasted = forecast[['ds','trend', 'yhat', 'yhat_lower', 'yhat_upper', 'fact']].copy()
forecasted['anomaly'] = 0
   forecasted.loc[forecasted['fact'] > forecasted['yhat_upper'], 'anomaly'] = 1
   forecasted.loc[forecasted['fact'] < forecasted['yhat_lower'], 'anomaly'] = -1
#anomaly importances
   forecasted['importance'] = 0
   forecasted.loc[forecasted['anomaly'] ==1, 'importance'] =
       (forecasted['fact'] - forecasted['yhat_upper'])/forecast['fact']
   forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] =
       (forecasted['yhat_lower'] - forecasted['fact'])/forecast['fact']

   return forecasted
pred = detect_anomalies(pred)
~~~

이를 시각화하면 아래와 같습니다.

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-detection.png?ssl=1)

</br></br>

**장점**</br>
이 방법은 월간, 연간과 같은 다양한 계절성 매개변수에 대해서 훌륭한 결과를 보여줍니다. Isolation Forest와 비교한다면 엣지 케이스에 대한 처리가 좋습니다.

</br>
**단점**</br>
이 방법은 예측을 기반으로 하기 때문에 제한된 데이터를 갖고 있는 경우 어려움이 있습니다. 제한된 데이터는 곧 예측 품질이 낮아지고 이상 감지 정확도가 낮이지기 떄문입니다.

</br></br></br>

### 클러스터링 기반 이상 탐지
클러스트링 기반 이상 탐지는 Isolation Forest와 같은 비지도 학습 기술이다.

간단하게 정의된 클러스터(집단) 외부에 있는 데이터를 잠재적인 이상치라 정의합니다.

k-mean 기법으로 분류를 해볼텐데 그러기 위해서 우선 처리할 클러스터 수를 알아야 합니다. 이때 Elbow Method가 매우 효율적입니다.

Elbow Method는 클러스터 수 vs 분산 설명/목표/점수의 그래프입니다.

~~~
data = df[['price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]
n_cluster = range(1, 20)
kmeans = [KMeans(n_clusters=i).fit(data) for i in n_cluster]
scores = [kmeans[i].score(data) for i in range(len(kmeans))]
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(n_cluster, scores)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show();
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Elbow-method.png?ssl=1)

10개의 군집에서 어느정도 안저적인 수평을 형성합니다. 따라서 클러스터 수를 10개로 결정합니다.

다음은 유지할 피처를 선정해야합니다.

~~~
data = df[['price_usd', 'srch_booking_window', 'srch_saturday_night_bool']]
X = data.values
X_std = StandardScaler().fit_transform(X)
#Calculating Eigenvecors and eigenvalues of Covariance matrix
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs.sort(key = lambda x: x[0], reverse= True)
# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) # Cumulative explained variance
plt.figure(figsize=(10, 5))
plt.bar(range(len(var_exp)), var_exp, alpha=0.3, align='center', label='individual explained variance', color = 'y')
plt.step(range(len(cum_var_exp)), cum_var_exp, where='mid',label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show();
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-detection-components.png?ssl=1)

첫 피처가 분산의 50%를 설명해주고 있습니다. 두번째는 30%넘게 설명하고 있습니다. 첫 2개의 피처가 80%의 정보를 포함하고 있습니다. 따라서 n_components를 2로 설정합니다. 여기서 중요한 점은 무시할만한 피처는 없다는 것을 잊어서 안됩니다.

클러스터링 기반의 이상 탐지의 기본 가정은 데이터를 클러스팅 한 결과에서 정상 데이터는 클러스터에 속하고 이상치는 어떤 클러스테에도 속하지 않거나 작은 클러스터에 속한다는 것입니다.

아래와 같은 방법으로 시각화를 합니다.
1. 각 점과 가장 가까운 중심 사이의 거리를 계산 합니다. 가장 큰 거리는 이상치로 간주합니다.
2. Isolation Forest와 유사하게 데이터셋에 존재하는 이상값의 비율에 대한 정보를 제공해야합니다.(outliers_fraciton) 이 하이퍼파라미터는 hit/trial 혹은 grid-search로 설정해야하는 변수입니다. 보통 시작 수치는 0.1로 추정합니다.
3. outliers_fraction을 사용하여 이상치 수를 계산합니다.
4. 임계값(threshold)를 이상치의 최소 거리로 설정합니다.
5. 'anomaly1'에 결과는 0:정상, 1:이상치로 나타납니다.


~~~
# return Series of distance between each point and its distance with the closest centroid
def getDistanceByPoint(data, model):
   distance = pd.Series()
   for i in range(0,len(data)):
       Xa = np.array(data.loc[i])
       Xb = model.cluster_centers_[model.labels_[i]-1]
       distance.at[i]=np.linalg.norm(Xa-Xb)
   return distance
outliers_fraction = 0.1
# get the distance between each point and its nearest centroid. The biggest distances are considered as anomaly
distance = getDistanceByPoint(data, kmeans[9])
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
# anomaly1 contain the anomaly result of the above method Cluster (0:normal, 1:anomaly)
df['anomaly1'] = (distance >= threshold).astype(int)
fig, ax = plt.subplots(figsize=(10,6))
colors = {0:'blue', 1:'red'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["anomaly1"].apply(lambda x: colors[x]))
plt.xlabel('principal feature1')
plt.ylabel('principal feature2')
plt.show();
~~~

이상치로 분류된 데이터를 실제값과 비교해봅니다.

~~~
df = df.sort_values('date_time')
fig, ax = plt.subplots(figsize=(10,6))
a = df.loc[df['anomaly1'] == 1, ['date_time', 'price_usd']] #anomaly
ax.plot(pd.to_datetime(df['date_time']), df['price_usd'], color='k',label='Normal')
ax.scatter(pd.to_datetime(a['date_time']),a['price_usd'], color='red', label='Anomaly')
ax.xaxis_date()
plt.xlabel('Date Time')
plt.ylabel('price in USD')
plt.legend()
fig.autofmt_xdate()
plt.show()
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Visualize-anomalies-dataframe.png?ssl=1)

결과는 약간의 실측이 있었지만 peak(꼭지점)의 값들을 잘 잡아냈습니다. 이런 문제의 일부는 outlier_fraction에 대한 값을 수정하지 않았기 때문일 수 있습니다.

</br></br>

**장점**
이 기술의 가장 큰 장점은 다른 비지도 학습과 비슷하게 더욱 정교한 모델을 만들기 위해 많은 피처들을 사용할 수 있습니다.

**단점**
피처의 수가 늘어날 수록 컴퓨터의 성능에 빠르게 영향을 줄 수 있습니다. 게다가 조정해야할 하이퍼파라미터들이 많기 때문에 성능에 대한 변동 가능성이 높습니다.

</br></br></br>

### 오토인코더(Autoencoders)
딥러닝 기술인 오토인코더를 사용하여 이상치 처리하는 방법이 있습니다.

오토인코더는 다른 차원을 통해 피처를 추출하면서 입력 데이터를 재생성하는 비지도 학습 기술입니다. 다시말해 오토인코더의 데이터 형태인 Latent Representation을 사용하여 차원 축소하는 것입니다.

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-detection-autoencoders.png?ssl=1)

**이상탐지에 차원 축소를 하는 이유**</br>
차원을 줄이면 이상치를 포함한 일부 정보가 손실되는것이 아닌가라는 의문이들 수 있습니다. 하지만 데이터의 주패턴이 식별되면 이상값은 자연스럽게 드러난다는 것입니다. 많은 거리 기반의 기술(ex. Knn)은 모든 피처 차원에서 모든 데이터의 거리를 계산할때 차원의 저주에 고통받습니다. 따라서 높은 차원을 줄여야 합니다.

흥미롭게도 차원을 축소하는 과정에서 이상치가 식별됩니다. 이상치 감지는 차원 축소의 부산물이라 할 수 있습니다.

오토인코더는 이상치를 찾는 비지도 학습 접근법입니다.

</br>


**왜 오토인코더인가?**</br>
이상치 탐지를 위한 PCA(Principal Component Analysis)가 있지만 왜 오토인코더가 필요할까요? 그 이유는 PCA는 선형 대수를 사용하여 변환하기 때문입니다. 대조적이게 오토인코더는 비선형 활성화 함수와 다중 계층을 사용하여 비선형 변환을 수행합니다. PCA로 하나의 거대한 변환으로 훈련시키는 것보다 오토인코더로 여러 계층으로 훈련시키는 것이 더 효율 적입니다. 오토인코더는 데이터 문제가 본질적으로 복잡하고 비선형일때 장점을 보여줍니다.

이제 오토인코더를 활용한 모델을 구축해봅시다.
'Tensorflow'와 'Pytorch'를 활용하여 오토인코더를 구현할 수 있습니다. 이번에는 간단한 구현이 목적이기에 PyOD라는 모듈을 사용하여 약간의 입력으로 오토인코더를 구현해봅니다.


~~~
import numpy as np
import pandas as pd
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import generate_data
contamination = 0.1  # percentage of outliers
n_train = 500  # number of training points
n_test = 500  # number of testing points
n_features = 25 # Number of features
X_train, y_train, X_test, y_test = generate_data(
   n_train=n_train, n_test=n_test,
   n_features= n_features,
   contamination=contamination,random_state=1234)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
~~~

비지도 학습을 처리할때 항상 표준화를 해주는 것이 안전한 방법입니다.

~~~
from sklearn.preprocessing import StandardScaler
X_train = StandardScaler().fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_test = StandardScaler().fit_transform(X_test)
X_test = pd.DataFrame(X_test)
~~~


훈련시킬 값들을 PCA로 2차원으 플로팅시켜 시각화 해봅니다.

~~~
from sklearn.decomposition import PCA
pca = PCA(2)
x_pca = pca.fit_transform(X_train)
x_pca = pd.DataFrame(x_pca)
x_pca.columns=['PC1','PC2']
cdict = {0: 'red', 1: 'blue'}
# Plot
import matplotlib.pyplot as plt
plt.scatter(X_train[0], X_train[1], c=y_train, alpha=1)
plt.title('Scatter plot')
plt.xlabel('x')
plt.ylabel('y')
plt.show()
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-detection-scatter-plot.png?ssl=1)

검은색이 정상값, 노란색이 이상치입니다.

1. 모델 구축
~~~
clf = AutoEncoder(hidden_neurons =[ 25 , 2 , 2 , 25 ])
clf.fit(X_train)
~~~

2. cut point 설정
훈련된 모델을 통해 테스트 데이터에서 이상치 점수를 예측해봅니다. 여기서 이상치 점수는 멀리 떨어진 거리로 계산이됩니다. PyOD함수의 .decision_function()은 각 데이터 지점에 대한 거리(이상치 점수)를 계산합니다.

~~~
# Get the outlier scores for the train data
y_train_scores = clf.decision_scores_
# Predict the anomaly scores
y_test_scores = clf.decision_function(X_test)  # outlier scores
y_test_scores = pd.Series(y_test_scores)
# Plot it!
import matplotlib.pyplot as plt
plt.hist(y_test_scores, bins='auto')
plt.title("Histogram for Model Clf1 Anomaly Scores")
plt.show()
~~~

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-detection-histogram.png?ssl=1)

위 그림은 히스토그램을 통해 이상점수에 대한 빈도를 계산한 것입니다. 높은 점수들이 빈도가 적은것을 확인 할 수 있고 cut point를 4.0으로 선택하는 것이 합당할 것으로 보입니다.

3. 각 클러스테의 통계치 가져오기
cut point인 4.0 미만의 클러스터를 0, 4.0이상인 클러스터를 1로 할당합니다. 그룹화를 통해 클러스터별 요약 통계를 계산합니다. 

~~~
df_test = X_test.copy()
df_test['score'] = y_test_scores
df_test['cluster'] = np.where(df_test['score']<4, 0, 1)
df_test['cluster'].value_counts()
df_test.groupby('cluster').mean()
~~~

아래의 결과는 각 클러스터으 평균 변수 값을 나타냅니다. 높은 점수는 표준에서 멀리 떨어져 있음을 뜻합니다.

![img](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Anomaly-detection-cluster.png?ssl=1)

</br></br>

**장점**</br>
1. 자동인코더는 고차원 데이터를 쉽게 처리합니다.
2. 비선형 방법, 고차원 데이터 세트 내의 복잡한 패턴 파악이 가능합니다.

**단점**</br>
1. 딥 러닝 기반의 방법이기에 데이터가 적을 경우 큰 어려움이 있습니다.
2. 깊은 네트워크가 있는 빅데이터를 다루면 계산 비용이 상상 이상으로 커집니다.
